In [ ]:
#r "nuget: NuGet.Protocol, 6.2.1"
#r "nuget: Plotly.NET, 3.0.0"
#r "nuget: Plotly.NET.Interactive, 3.0.0"
#r "nuget: LibGit2Sharp, 0.26.2"
#r "nuget: BenchmarkDotnet, 0.13.1"
#r "../../../../artifacts/bin/HistoricalBenchmark.Runner/Release/net6.0/HistoricalBenchmark.Runner.dll"

open HistoricalBenchmark.Runner

let config =
    {
        RunConfig.Time = DateTime(2022, 7, 12, 8, 19, 0)
        // Set to a valid local absolute path
        BaseDir = "d:/benchmarks"
        Parallelism = 3
        ResultsSuffix = "results"
    }

In [ ]:
// let newerVersion = findVersion "40.0.1-preview.21352.5"
// let olderVersion = findVersion "40.0.0"
// let newerRevision = NuGet.resolvePackageRevision newerVersion
// let olderVersion = NuGet.resolvePackageRevision olderVersion

// Find all commits between the two
let newerRevision = "cb3692bf5d5af46b075c50d4f802f09152fb6b67"
let olderRevision = "4d330f37ae740adccd2586d568b6d45d2c408fad"
let cNewer, cOlder = Git.findCommit config newerRevision, Git.findCommit config olderRevision
let versions =
    Git.findCommitsBetweenInclusive config cOlder cNewer |> Seq.toList
    |> Git.excludeBadCommits ["679e11c533ad6c408723cf9b227a57be23fb51db"]
    |> List.map (fun c -> FCSVersion.Git c.Sha)
// Fetch, build and run benchmarks for all of them
Runner.runAll {config with ResultsSuffix = "between_2_nuget_versions"} versions

In [ ]:
// Find two latest NuGet versions of FCS (including prereleases), run benchmarks and export results
NuGet.allVersionsMetadata.Value()
|> List.take 10
|> List.map (fun metadata -> FCSVersion.NuGet metadata.Identity.Version)
|> Runner.runAll {config with ResultsSuffix = "10_latest_nuget_versions"}

In [ ]:
let versions =
    [
        FCSVersion.Local
        FCSVersion.Git "4edcbb45365f96d4858f438a556b93d4a32ae219"
        FCSVersion.NuGet (NuGet.findVersion "41.0.5-preview.22329.3")
        FCSVersion.NuGet (NuGet.findVersion "41.0.5")
        FCSVersion.NuGet (NuGet.findVersion "40.0.0")
        FCSVersion.NuGet (NuGet.findVersion "35.0.0")
        FCSVersion.NuGet (NuGet.findVersion "30.0.0")
        FCSVersion.NuGet (NuGet.findVersion "25.0.1")
        FCSVersion.NuGet (NuGet.findVersion "20.0.1")
        FCSVersion.NuGet (NuGet.findVersion "13.0.0")
    ]
    
Runner.runAll {config with ResultsSuffix = "sample_versions"} versions